# download genome info 
using assembly accession from NCBI

In [ ]:
from grab_genomes_by_accession import download_genome

genome_database_dir = rf'G:\My Drive\Dar lab\code\probe_design\methods\genomes'
accession_list = ['ASM584v2'] #['ASM904v1:sal',"ASM584v2:ecoli","'ViralProj14493'"]

for accession in accession_list:
    download_genome(accession,genome_database_dir)


# generate probes 
define a set of assemblies to work with
generate a single probe design for all

In [ ]:
from probe_designer import design_probes

genomes_dir = rf'G:\My Drive\Dar lab\code\probe_design\methods\genomes'
output_dir = rf'G:\My Drive\Dar lab\code\probe_design\methods\probes'
accession_list = ['ASM584v2','ViralProj14493'] #adding negative controls #"ASM584v2" ASM904v1
run_name = 'ecoli_k12'
run_dir = rf'{output_dir}/{run_name}'

design_probes(genomes_dir,
              accession_list,
              output_dir,
              run_name,
              probe_props=None,
              max_nonspecific_match=18,
              is_allow_duplicates = True, min_pid=95, min_len_similarity_percent=90, min_percent_cov=90 #genes 95% identical with 90% cov
              )



In [ ]:
from probe_designer import calculate_probes_per_gene

output_dir = rf'G:\My Drive\Dar lab\code\probe_design\methods\probes'
run_name = 'ecoli_k12'
run_dir = rf'{output_dir}/{run_name}'

calculate_probes_per_gene(run_dir,min_distance = 0, overlap = 5, method = 'dp')

# select genes

generate semi randomly selected gene sets

include or exclude genes

# sample the genome

hold out on references + ensure various genes will be selected

In [ ]:
# what should be the set size?
set_size = 138
ref_size = 12# 9 genes, 3xdnaA,  negatives not counted
num_of_sets = 16
num_of_hybs = (set_size+ref_size)/3
num_of_probes = num_of_hybs*(16 + 18 + 26)
total_genes = set_size*num_of_sets + ref_size

num_of_conditions = 8
min_cells_per_cond = 2000
total_cells = num_of_conditions*num_of_sets*min_cells_per_cond
cells_per_fov = 10000
fovs_needed = total_cells/cells_per_fov
print(total_genes,num_of_probes,total_cells,fovs_needed)

In [ ]:
from gene_selection import sample_genes_for_par2fish

probe_dir = r'G:\My Drive\Dar lab\code\probe_design\methods\probes\ecoli_k12'
probe_sum_table = 'probes_per_gene_summary_dp_min_0_overlap_5.txt'
biocyc_tu_table = 'biocyc_k12_genes_and_TUs.txt' #manually downloaded
gene_set_output_dir = rf'{probe_dir}\gene_sets' #where to save the output

#params 
set_size = 138
num_of_sets = 16
min_probe_per_gene = 8
desired_probe_count = 26
max_double_operons = 50

### genes to avoid - for adding as the reference between libraries
reference_genes = { #must 
    'cre' : 'P1_gp003', #negative controls
    'pro' : 'P1_gp013', 
    'dbn' : 'P1_gp078', 
    'dnaA' : 'b3702', #3 channel controls
    'dnaN' : 'b3701',
    'rpoS':'b2741', # stationary phase marker
    'rpsC' : 'b3314', #biomass
    'recA':'b2699', #SOS response - known heterogeneity
    'fliC':'b1923', #flagella, known heterogeneity
    'gadB' : 'b1493',#acid stress - known hyper variable
    'fimA' : 'b4314', # fimbria
    'cheA' : 'b1888', # chemotaxis
    'slp' : 'b3506' # highly variable starvation lipoprotein
}

# genes to ensure are in list (will appear in lib 1 or spill over if set_size is smaller)
genes_of_interest_for_lib_1 = {'hipA':'b1507', #toxin of persister causing TA system
                               'csgA':'b1042', #curli major subunit
                               'pitB':'b2987', #metal_phosphate:H(+)_symporter
                               'hslJ':'b1379', #lipoprotein_implicated_in_Novobiocin_resistance
                               'pyrB' : 'b4245', #aspartate_carbamoyltransferase_catalytic_subunit
                               'lptD' : 'b0054',
                               'lacZ' : 'b0344', #beta-galactosidase
                               'treF' : 'b3519',
                               'hdeD' : 'b3511',
                               'pdxI' : 'b1406',
                               'norV' : 'b2710',
                               'carA' : 'b0032',
                               'argB' : 'b3959'
                               }

#sample the genes and divide into text files for the probe assebler stage
sampler = sample_genes_for_par2fish(probe_dir = probe_dir,
                                    probe_sum_table = probe_sum_table,
                                    biocyc_tu_table = biocyc_tu_table,
                                    set_size = set_size, 
                                    num_of_sets = num_of_sets, 
                                    min_probe_per_gene = min_probe_per_gene, 
                                    desired_probe_count = desired_probe_count,
                                    genes_of_interest = list(genes_of_interest_for_lib_1.values()),
                                    genes_to_avoid = list(reference_genes.values()),
                                    max_double_operons = max_double_operons,
                                    output_dir = gene_set_output_dir
                                    )


# generate order-ready sequences 
Design:
- dnaA-dnaN 3 channel control: measure efficiency 
- phage negative controls
- 9 reference genes for library efficiency + testing the concept of targeted mode (var genes of interest such as fliC)
- Targted mode: will use available ROs or design new ones

- known hyper variable genes in pool 1 (collected from reporter assay papers)
- set completed to 2220 genes


In [ ]:
from probe_assembler import ProbeAssembler
import pandas as pd

#exp params
experiment_dir = r'G:\My Drive\Dar lab\code\probe_design\methods\probes\ecoli_k12'
output_dir = rf'{experiment_dir}/assembled_probe_sets'
set_size = 138
num_of_sets = 16

#reference peppered in top of list for 
# must be in multiples of 3 or complete the set size to a num divisible by 3
reference_genes = {
    'rpoS':'b2741', # stationary phase marker
    'ackA' : 'b2296', # acetate fermentation
    'rpsC' : 'b3314', # biomass marker
    'recA':'b2699', # SOS response - known heterogeneity
    'fliC':'b1923', # flagella, known heterogeneity
    'gadB' : 'b1493',# acid stress - known hyper variable
    'fimA' : 'b4314', # fimbria
    'cheA' : 'b1888', # chemotaxis
    'slp' : 'b3506' # highly variable starvation lipoprotein
}

# initiate the assembler
probe_designer = ProbeAssembler(
    probe_per_gene_file = rf'{experiment_dir}/probes_per_gene_summary_dp_min_0_overlap_5.txt', #
    primary_probe_path = rf'{experiment_dir}/specific_probes_dp_min_0_overlap_5.txt', # the probes
    path_to_gene_list = rf'{experiment_dir}/gene_sets/gene_set_1_{set_size}_genes.txt', #an excel with locus_tag column
    output_dir = output_dir,
    is_amplification= True, #True if using Twist
    primer_set = 1, # choose 1 or 4 (better 1)
    num_of_flanking = 2, #choose 2 or 4 
    max_probes_per_ch={'A647':16,'A550':18,'A488':30}, # up to how many probes per channel?
    probes_to_remove = ['R6','R21','R36','R107'], # problematic ROs to remove
    reference_genes = list(reference_genes.values()),
    genes_of_interest=[], #genes to priorize in channel A647
    positive_control_genes = { 'dnaA-dnaN': {'genes': ['b3702','b3701'], 'readouts' : {'A647': 4,'A488' : 5, 'A550' : 9} } }, # gene or operon where probes for 3 channels will be generated
    negative_control_genes = { 'genes': ['P1_gp003','P1_gp013','P1_gp078'], 'readouts' : {'A647': 1,'A488' : 2, 'A550' : 3} }, # gene or operon where probes for 3 channels will be generated #'genes': ['P1_gp003','P1_gp013','P1_gp078'], 'readouts' : {'A647': 4,'A488' : 5, 'A550' : 15}
    fwd_linker = 'TT' # if using USER enzyme then TT, else AA
)

# run for each lib and combine for order
probe_count = 0
probe_set = pd.DataFrame()
for set_idx in range(1,num_of_sets+1):
    probe_designer.path_to_gene_list = rf'{experiment_dir}\gene_sets\gene_set_{set_idx}_{set_size}_genes.txt'
    probe_designer.set_num = set_idx
    if set_idx % 2 == 0:
        probe_designer.reset_primer_set(primer_set=4)
    else:
        probe_designer.reset_primer_set(primer_set=1)
    probe_designer.design()
    probe_designer.probe_set['probe_set'] = set_idx
    probe_set = pd.concat([probe_set,probe_designer.probe_set]) #add probes of lib 1 and 2
    
    probe_count = probe_designer.total_probes
    print(set_idx,probe_count)
 


In [ ]:
import pandas as pd

experiment_dir = r'G:\My Drive\Dar lab\code\probe_design\methods\probes\ecoli_k12'

set_size = 138

#organize pairs so that they are <=6000 probes 
pairs = [(1,4),(13,2),(3,6),(5,8),(7,10),(9,12),(11,14),(15,16)]

for pair in pairs:
        idxA,idxB = pair[0],pair[1]
        probe_set_A = pd.read_csv(rf'{experiment_dir}\assembled_probe_sets\probes_set_{idxA}_primers_1.txt',sep='\t')
        probe_set_B = pd.read_csv(rf'{experiment_dir}\assembled_probe_sets\probes_set_{idxB}_primers_4.txt',sep='\t')
        merged_probes = pd.concat([probe_set_A,probe_set_B])
        print('pair = ',pair,'total probes = ',len(merged_probes))
        merged_probes.to_csv(rf'{experiment_dir}\assembled_probe_sets\merged_sets_{idxA}_and_{idxB}.txt',sep='\t',index=False)

        #twist file
        twist_order = pd.DataFrame(columns=['probe_id','probe_seq'])
        twist_order['probe_seq'] = merged_probes['full_probe_seq']
        twist_order['probe_id'] = [f'probe_{i}' for i in range(1,len(merged_probes)+1)]
        twist_order.to_excel(rf'{experiment_dir}\assembled_probe_sets\twist_order_sets_{idxA}_and_{idxB}.xlsx')
        